If you would rather *read* this, you can find it on [LearnByBuilding.AI](https://learnbybuilding.ai/?ref=lbb-notebook). This notebook only contains code, to get some prose along with it, check out the tutorial posted there.

If you like this content, [follow me on twitter](https://twitter.com/bllchmbrs) and [sign up for the newsletter](https://learnbybuilding.ai/newsletter?ref=lbb-notebook). I'm posting all week about DSPy and providing a lot of "hard earned" lessons that I've gotten from learning how to build with AI from RAG, to DSPy and everything in between.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import dspy
turbo = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=1000, api_base="http://0.0.0.0:4000")
dspy.settings.configure(lm=turbo)

In [4]:
context = "Provide a greeting!"

In [5]:
v1 = dspy.Predict("context -> greeting")
print(v1)

Predict(StringSignature(context -> greeting
    instructions='Given the fields `context`, produce the fields `greeting`.'
    context = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Context:', 'desc': '${context}'})
    greeting = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Greeting:', 'desc': '${greeting}'})
))


In [6]:
print(v1.forward(context=context).greeting)

Hello! How can I assist you today?


In [7]:
class MakeGreeting(dspy.Module):
    def __init__(self, invalid_greetings = []):
        self.invalid_greetings = invalid_greetings
        self.prog = dspy.ChainOfThought("context -> greeting")

    def forward(self, context):
        return self.prog(context=context)

In [8]:
MakeGreeting().forward(context)

Prediction(
    rationale='produce the greeting. We need to consider the time of day, the formality of the situation, and the relationship between the speaker and the listener.',
    greeting='Hello!'
)

In [9]:
class MakeGreeting2(dspy.Module):
    def __init__(self, invalid_greetings = []):
        self.invalid_greetings = invalid_greetings
        self.prog = dspy.ChainOfThought("context -> greeting")

    def forward(self, context):
        result = self.prog(context=context)
        _greeting = result.greeting
        print(_greeting)
        greeting_violations = list(filter(lambda x: x.lower() in _greeting.lower(), self.invalid_greetings))
        print(greeting_violations)
        formatted = ", ".join(greeting_violations)
        dspy.Suggest(not bool(greeting_violations), f"Greetings like {formatted} are so bad, provide a different greeting.")
        return result

In [10]:
g2 = MakeGreeting2(invalid_greetings=['hello']).activate_assertions()
g2.forward(context)

ERROR:dspy.primitives.assertions:2024-05-21T17:45:57.966687Z [error    ] SuggestionFailed: Greetings like hello are so bad, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=111


Hello!
['hello']
Good morning!
[]


Prediction(
    greeting='Good morning!'
)

In [11]:
turbo.inspect_history()




Given the fields `context`, produce the fields `greeting`.

---

Follow the following format.

Context: ${context}

Past Greeting: past output with errors

Instructions: Some instructions you must satisfy

Greeting: ${greeting}

---

Context: Provide a greeting!

Past Greeting: Hello!

Instructions: Greetings like hello are so bad, provide a different greeting.

Greeting: Good morning!





'\n\n\nGiven the fields `context`, produce the fields `greeting`.\n\n---\n\nFollow the following format.\n\nContext: ${context}\n\nPast Greeting: past output with errors\n\nInstructions: Some instructions you must satisfy\n\nGreeting: ${greeting}\n\n---\n\nContext: Provide a greeting!\n\nPast Greeting: Hello!\n\nInstructions: Greetings like hello are so bad, provide a different greeting.\n\nGreeting:\x1b Good morning!\x1b\n\n\n'

In [12]:
class MakeGreeting3(dspy.Module):
    def __init__(self, invalid_greetings = []):
        self.invalid_greetings = invalid_greetings
        self.prog = dspy.ChainOfThought("context -> greeting")
        self.prev_greetings = []

    def forward(self, context):
        result = self.prog(context=context)
        self.prev_greetings.append(result.greeting)
        _greeting = result.greeting
        print(_greeting)
        greeting_violations = list(filter(lambda x: x.lower() in _greeting.lower(), self.invalid_greetings))
        print(greeting_violations)
        formatted = ", ".join(greeting_violations)
        dspy.Assert(not bool(greeting_violations), f"Greetings like {formatted} are so bad, provide a different greeting.")
        
        return result

In [13]:
g3 = MakeGreeting3(invalid_greetings=['hello']).activate_assertions()

g3.forward(context)

ERROR:dspy.primitives.assertions:2024-05-21T17:45:59.082076Z [error    ] AssertionError: Greetings like hello are so bad, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=88


Hello!
['hello']
Good morning!
[]


Prediction(
    greeting='Good morning!'
)

In [14]:
class MakeGreeting4(dspy.Module):
    def __init__(self, invalid_greetings = []):
        self.invalid_greetings = invalid_greetings
        self.prog = dspy.ChainOfThought("context -> greeting")
        self.prev_greetings = []

    def forward(self, context):
        result = self.prog(context=context)
        self.prev_greetings.append(result.greeting)
        _greeting = result.greeting
        print(_greeting)
        greeting_violations = list(filter(lambda x: x.lower() in _greeting.lower(), self.invalid_greetings))
        print(greeting_violations)
        formatted = ", ".join(greeting_violations)
        formatted_prev = ", ".join(self.prev_greetings)
        dspy.Suggest(not bool(greeting_violations), f"Greetings like {formatted} are so bad, provide a different greeting.")
        dspy.Suggest(not _greeting in self.prev_greetings, f"You've already used the greetings: {formatted_prev}, provide a different greeting.")
        
        return result

In [15]:
mg4 = MakeGreeting4(invalid_greetings=['hello']).activate_assertions()
mg4.forward(context)

ERROR:dspy.primitives.assertions:2024-05-21T17:45:59.873222Z [error    ] SuggestionFailed: Greetings like hello are so bad, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR:dspy.primitives.assertions:2024-05-21T17:45:59.875670Z [error    ] SuggestionFailed: You've already used the greetings: Hello!, Good morning!, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR:dspy.primitives.assertions:2024-05-21T17:45:59.882794Z [error    ] SuggestionFailed: You've already used the greetings: Hello!, Good morning!, Good afternoon!, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=108


Hello!
['hello']
Good morning!
[]
Good afternoon!
[]


Prediction(
    greeting='Good afternoon!'
)

In [16]:
mg4.forward(context)

ERROR:dspy.primitives.assertions:2024-05-21T17:46:00.327864Z [error    ] SuggestionFailed: Greetings like hello are so bad, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR:dspy.primitives.assertions:2024-05-21T17:46:00.332009Z [error    ] SuggestionFailed: You've already used the greetings: Hello!, Good morning!, Good afternoon!, Hello!, Good morning!, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR:dspy.primitives.assertions:2024-05-21T17:46:00.339213Z [error    ] SuggestionFailed: You've already used the greetings: Hello!, Good morning!, Good afternoon!, Hello!, Good morning!, Good day!, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=108


Hello!
['hello']
Good morning!
[]
Good day!
[]


Prediction(
    greeting='Good day!'
)

In [17]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from functools import partial

one_retry = partial(backtrack_handler, max_backtracks=1)
g4_with_assert_1_retry = assert_transform_module(MakeGreeting4(), one_retry)

In [18]:
g4_with_assert_1_retry.forward(context)

ERROR:dspy.primitives.assertions:2024-05-21T17:46:03.433354Z [error    ] SuggestionFailed: You've already used the greetings: Hello!, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR:dspy.primitives.assertions:2024-05-21T17:46:03.440421Z [error    ] SuggestionFailed: You've already used the greetings: Hello!, Hi there!, provide a different greeting. [dspy.primitives.assertions] filename=assertions.py lineno=108


Hello!
[]
Hi there!
[]


Prediction(
    greeting='Hi there!'
)

Follow along for subsequent tutorials on:

1. Automatically optimizing prompts
2. Customizing input to DSPy
3. Saving prompts to use in LangChain or LlamaIndex
4. Tuning and using open source models

Cheers,
[Bill](https://twitter.com/bllchmbrs) - [Learn By Building AI](https://learnbybuilding.ai/?ref=lbb-notebook)